# Document loaders

We will examine how to load and split popular file formats. The purpose of is to insert these documents into a vector database for semantic searches.

In [1]:
# Download the NLTK data
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/kenneth/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/kenneth/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

NLTK (Natural Language Toolkit) datasets are collections of linguistic data used for Natural Language Processing (NLP) tasks. Here's what they're used for:

## Setup: Download NLTK Data

Download required NLTK datasets for text processing:
- `punkt_tab` - Sentence tokenization
- `averaged_perceptron_tagger_eng` - Part-of-speech tagging

In [2]:
# Imports
import json, random
from langchain_community.document_loaders import UnstructuredEPubLoader, UnstructuredExcelLoader, UnstructuredPowerPointLoader
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, JSONLoader, UnstructuredXMLLoader
from langchain_community.document_loaders import UnstructuredEmailLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, RecursiveJsonSplitter


## Import Libraries

Import all necessary libraries for document loading and text splitting:
- **Document Loaders**: Various loaders for different file formats (EPUB, Excel, PowerPoint, PDF, DOCX, JSON, XML, Email)
- **Text Splitters**: Tools to split documents into smaller chunks for vector database insertion

# Create a text splitter

The purpose of splitting the text is to enable more precise search results by ensuring that each smaller segment captures relevant context within a larger document. When these text fragrments are used as context in prompts, they will not exceed the context window.

In [3]:
# TODO: Create a text splitter
chunk_size = 300
chunk_overlap = 30

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

## Loading and splitting documents

Documents can be loaded and split either as a single or as elements eg pages, individual slides, spreadsheet, etc. Set the `mode` arguemnt to
- `single` - load the entire document as a single object default for most loaders
- `multi`, `elements` - split the document into their respective elements eg pages, slides, etc.

In [5]:
def print_chunk_info(chunks):
   print(f'No of chunks: {len(chunks)}')
   idx = random.randrange(0, len(chunks))
   print(f'Chunk index: {idx}')
   print('Chunk details')
   for k, v in enumerate(chunks[idx]):
      print(f'\t{k} = {v}')

## Utility Function: Print Chunk Information

This helper function displays:
- Total number of chunks created
- A random chunk sample with its metadata
- Useful for inspecting how documents are being split

In [6]:
# TODO: Word doucment
# create a loader
word_loader = Docx2txtLoader('./docs/SST RL Python Setup.docx')
chunks = word_loader.load_and_split(text_splitter)
print_chunk_info(chunks)

No of chunks: 12
Chunk index: 8
Chunk details
	0 = ('id', None)
	1 = ('metadata', {'source': './docs/SST RL Python Setup.docx'})
	2 = ('page_content', 'After you have installed VSC, launch it. Next, we need to install the Python Extension for VSC. Open the extension panel by clicking on this icon  on the menu along VSC’s left window edge. Type Python on the extension window and select the Microsoft Python Extension. See the following image')
	3 = ('type', 'Document')


### Loading Word Documents (.docx)

Uses `Docx2txtLoader` to:
- Load Microsoft Word documents
- Split into chunks using the text splitter
- Extract text content while preserving basic formatting

In [7]:
# TODO: Excel doucment

# Load as a single document
xls_loader = UnstructuredExcelLoader('./docs/Financial Sample.xlsx')
chunks = xls_loader.load_and_split(text_splitter)

print_chunk_info(chunks)

No of chunks: 298
Chunk index: 253
Chunk details
	0 = ('id', None)
	1 = ('metadata', {'source': './docs/Financial Sample.xlsx'})
	2 = ('page_content', 'High 1249 10 20 24980 3247.4 21732.6 12490 9242.6 2014-10-01 00:00:00 10 October 2014 Government Germany Paseo High 357 10 350 124950 16243.5 108706.5 92820 15886.5 2014-11-01 00:00:00 11 November 2014 Channel Partners Germany Paseo High 1013 10 12 12156 1580.28 10575.72 3039 7536.72 2014-12-01')
	3 = ('type', 'Document')


### Loading Excel Documents (.xlsx) - Single Mode

Uses `UnstructuredExcelLoader` with default settings:
- Loads entire spreadsheet as a single document
- Useful when all data should be treated as one context
- Preserves cell values and basic structure

In [8]:
# "text_as_html" key in the document metadata.
xls_loader = UnstructuredExcelLoader('./docs/Financial Sample.xlsx', mode="elements")
chunks = xls_loader.load_and_split(text_splitter)

print_chunk_info(chunks)

No of chunks: 298
Chunk index: 232
Chunk details
	0 = ('id', None)
	1 = ('metadata', {'source': './docs/Financial Sample.xlsx', 'file_directory': './docs', 'filename': 'Financial Sample.xlsx', 'last_modified': '2025-11-25T13:34:12', 'page_name': 'Sheet1', 'page_number': 1, 'text_as_html': '<table><tr><td>Segment</td><td>Country</td><td>Product</td><td>Discount Band</td><td>Units Sold</td><td>Manufacturing Price</td><td>Sale Price</td><td>Gross Sales</td><td>Discounts</td><td>Sales</td><td>COGS</td><td>Profit</td><td>Date</td><td>Month Number</td><td>Month Name</td><td>Year</td></tr><tr><td>Government</td><td>Canada</td><td>Carretera</td><td/><td>1618.5</td><td>3</td><td>20</td><td>32370</td><td>0</td><td>32370</td><td>16185</td><td>16185</td><td>2014-01-01 00:00:00</td><td>1</td><td>January</td><td>2014</td></tr><tr><td>Government</td><td>Germany</td><td>Carretera</td><td/><td>1321</td><td>3</td><td>20</td><td>26420</td><td>0</td><td>26420</td><td>13210</td><td>13210</td><td>2014-01-01

### Loading Excel Documents - Elements Mode

Same Excel file loaded with `mode="elements"`:
- Splits spreadsheet into individual elements (rows, cells, tables)
- Metadata includes HTML representation (`text_as_html`)
- Better for granular analysis of spreadsheet data

In [9]:
# TODO: Load Powerpoint

# Load as a single document
ppt_loader = UnstructuredPowerPointLoader('./docs/The Gettysburg Address.ppt.pptx')
chunks = ppt_loader.load_and_split(text_splitter)

print_chunk_info(chunks)


No of chunks: 37
Chunk index: 36
Chunk details
	0 = ('id', None)
	1 = ('metadata', {'source': './docs/The Gettysburg Address.ppt.pptx'})
	2 = ('page_content', 'Why would this place be so special?\n\n\n\nCritical Thinking Discussion\n\nToward the end of the speech, Lincoln talks about the work which the dead an injured so “nobly” advanced.  \n\nHow can war be noble?')
	3 = ('type', 'Document')


### Loading PowerPoint Presentations (.pptx)

Uses `UnstructuredPowerPointLoader` to:
- Extract text from presentation slides
- Load as a single document
- Preserves text content from all slides sequentially

In [10]:
# TODO: Load Email

# Load as a single document
email_loader = UnstructuredEmailLoader('./docs/sample.eml')
chunks = email_loader.load_and_split(text_splitter)

print_chunk_info(chunks)


No of chunks: 2
Chunk index: 0
Chunk details
	0 = ('id', None)
	1 = ('metadata', {'source': './docs/sample.eml'})
	2 = ('page_content', "Testing Manuel Lemos' MIME E-mail composing and sending PHP class: HTML message Hello Manuel, This message is just to let you know that the MIME E-mail message composing and sending PHP class is working as expected. Here is an image embedded in a message as a separate part: Thank you, mlemos")
	3 = ('type', 'Document')


### Loading Email Files (.eml)

Uses `UnstructuredEmailLoader` to:
- Parse email message files
- Extract headers (from, to, subject, date)
- Extract email body content
- Useful for processing archived emails

In [11]:
# TODO: Load PDF
# Use password=<password> for password protected documents
pdf_loader = PyPDFLoader(file_path='./docs/Path-to-GitOps-Red-Hat-Developer-e-book.pdf', mode="page", extract_images=True)
chunks = pdf_loader.load_and_split(text_splitter)

print_chunk_info(chunks)

No of chunks: 406
Chunk index: 280
Chunk details
	0 = ('id', None)
	1 = ('metadata', {'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2022-07-15T19:22:48-05:00', 'moddate': '2022-07-17T10:32:28-04:00', 'title': 'The Path to GitOps', 'source': './docs/Path-to-GitOps-Red-Hat-Developer-e-book.pdf', 'total_pages': 45, 'page': 31, 'page_label': '32'})
	2 = ('page_content', 'Chapter 5 – Repository and Directory Structures\nThe Path to GitOps | 32\nrepository secrets, and configuration files specific to the Git controller, Argo CD. \nEach configuration has its own directory.\n•   core: This contains Y AML for the core functionality of the cluster. The Kubernetes')
	3 = ('type', 'Document')


### Loading PDF Documents (.pdf)

Uses `PyPDFLoader` to:
- Extract text from PDF files page by page (`mode="page"`)
- Extract embedded images (`extract_images=True`)
- Supports password-protected PDFs with `password` parameter
- Preserves page metadata for source tracking

In [ ]:
# TODO: Load EPUB

# Load as a single document
epub_loader = UnstructuredEPubLoader('./docs/the_happy_prince.epub')
chunks = epub_loader.load_and_split(text_splitter)

print_chunk_info(chunks)


### Loading EPUB eBooks (.epub)

Uses `UnstructuredEPubLoader` to:
- Extract text from electronic book files
- Preserves chapter structure and metadata
- Useful for processing digital books and publications

## Processing structured document/JSON

In [12]:
# TODO: Create a JSON splitter
json_splitter = RecursiveJsonSplitter(max_chunk_size=128)


### Create JSON Splitter

`RecursiveJsonSplitter` is designed for structured JSON data:
- `max_chunk_size=128` - Maximum size of each JSON chunk
- Recursively splits nested JSON structures
- Preserves JSON structure while staying within size limits

In [13]:
# TODO: Process a single JSON object 
with open('./docs/single.json') as f:
   json_data = json.load(f)

chunks = json_splitter.split_json(json_data=json_data)

print_chunk_info(chunks)

No of chunks: 4
Chunk index: 0
Chunk details
	0 = web-app


### Process Single JSON Object

Demonstrates processing a single JSON object:
- Loads JSON file into memory
- Uses `split_json()` to split the data structure
- Maintains JSON hierarchy while creating manageable chunks
- Returns list of JSON fragments

In [14]:
# TODO: Process an array of JSON objects, return chunks as Dictionary

with open('./docs/tv-shows.json') as f:
   json_data = json.load(f)

chunks = json_splitter.create_documents(texts=json_data)

print(chunks)

print_chunk_info(chunks)


[Document(metadata={}, page_content='{"id": 1, "url": "http://www.tvmaze.com/shows/1/under-the-dome", "name": "Under the Dome", "type": "Scripted"}'), Document(metadata={}, page_content='{"language": "English", "genres": ["Drama", "Science-Fiction", "Thriller"], "status": "Ended", "runtime": 60}'), Document(metadata={}, page_content='{"premiered": "2013-06-24", "officialSite": "http://www.cbs.com/shows/under-the-dome/"}'), Document(metadata={}, page_content='{"schedule": {"time": "22:00", "days": ["Thursday"]}, "rating": {"average": 6.5}, "weight": 91}'), Document(metadata={}, page_content='{"network": {"id": 2, "name": "CBS", "country": {"name": "United States", "code": "US", "timezone": "America/New_York"}}}'), Document(metadata={}, page_content='{"webChannel": null, "externals": {"tvrage": 25988, "thetvdb": 264492, "imdb": "tt1553656"}}'), Document(metadata={}, page_content='{"image": {"medium": "http://static.tvmaze.com/uploads/images/medium_portrait/0/1.jpg"}}'), Document(metadata

### Process Array of JSON Objects

Demonstrates processing JSON arrays:
- Loads JSON array from file
- Uses `create_documents()` to convert each object into a Document
- Returns LangChain Document objects with metadata
- Useful for datasets with multiple records

In [15]:
# TODO: Extract specific attributes from the JSON document, use JSON path to define which element

json_loader = JSONLoader(
   file_path='./docs/tv-shows.json',
   jq_schema='.[].summary',
   text_content=True
)

# don't split, just extract summary
chunks = json_loader.load()

print_chunk_info(chunks)

No of chunks: 240
Chunk index: 109
Chunk details
	0 = ('id', None)
	1 = ('metadata', {'source': '/home/kenneth/Projects/ELLM-workshop/ellm-2025-nov17/day03/docs/tv-shows.json', 'seq_num': 110})
	2 = ('page_content', '<p>Eighteen to Twenty castaways will compete against each other on <b>Survivor</b>, when the Emmy Award-winning series returns on Wednesdays on CBS Television Network. This edition of SURVIVOR, filmed in Cagayan, a province in the Philippines, will be themed "Brawn vs. Brains vs. Beauty." The groupings of these tribes, each comprised of six castaways, reflect the distinguishing qualities that people bring to the game and test whether there is any truth to the pre-conceived notion that that certain characteristics will help you win the game of SURVIVOR. All castaways will compete to outwit, outplay, outlast and ultimately be crowned Sole Survivor. The show is hosted by Emmy Award winner Jeff Probst.</p>')
	3 = ('type', 'Document')


### Extract Specific JSON Attributes with JSONPath

Uses `JSONLoader` with JQ schema for targeted extraction:
- `jq_schema='.[].summary'` - Extract only the `summary` field from each array element
- `text_content=True` - Treat extracted data as text
- No splitting applied - extracts complete summaries
- Useful for extracting specific fields from complex JSON structures